## 0. Environment Setup
We create a smiliar environemnt that simulate the inputs and variables as per in ciop jobs

In [22]:
#old input1='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7' #niger-hype-model.zip, one file when unzipped
#old input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
#use this from model.zip instead  input_hype_data='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2' #niger-hype-data.zip, dirs when unzipped
TMPDIR="/workspace/tmpdir"
TMPDIR

[1] "/workspace/tmpdir"

1. Read input model
This inital step collect via proper opensearch call and copy the HYPE model config archive

In [23]:
app.date = format(Sys.time(), "%Y%m%d_%H%M")

## set application name
app.name = "forecast"
## ------------------------------------------------------------------------------
## flag which environment is used, if not set
if(!exists("app.sys")){
    app.sys ="tep"
    }
## ------------------------------------------------------------------------------
## load rciop package and set working directory to TMPDIR when running on TEP 
if(app.sys=="tep"){
    library("rciop")

    rciop.log ("DEBUG", " *** hypeapps-forecast *** TEP hydrological modelling applications ***", "/node_forecast/run.R")
    rciop.log ("DEBUG", " rciop library loaded", "/node_forecast/run.R")

    setwd(TMPDIR)
    rciop.log("DEBUG", paste(" R session working directory set to ",TMPDIR,sep=""), "/node_forecast/run.R")
}

Some rciop functions may need to be overridden in the notebook environment.
Set to your current needs.

In [24]:
ENABLE_STUB_RCIOP_LOG <- FALSE
#ENABLE_STUB_RCIOP_LOG <- TRUE

if (ENABLE_STUB_RCIOP_LOG == TRUE) {
    message("rciop.log() overridden by local stub function...")
    
    rciop.log <- function(type, message, proc='') {

        type <- type
        msg <- message
    if (proc=='') { proc <- "user process" }

    out.msg <- paste(format(Sys.time(), format="%Y-%m-%dT%H:%M:%S"), "[", type, "][", proc, "] ", msg, sep="")

    message(out.msg)
    } # rciop.log

} # end stub

In [25]:
#ENABLE_STUB_RCIOP_GETPARAM <- FALSE
ENABLE_STUB_RCIOP_GETPARAM <- TRUE

if (ENABLE_STUB_RCIOP_GETPARAM == TRUE) {
    message("rciop.getparam() overridden by local stub function...")
    # Change according to your test objective.
    # Would normally return values defined in application.xml for specified ids'.
    
    rciop.getparam <- function(id) {
        value <- NULL # or ""
        
        if (id == "model") {
            value <- "https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7"
        }
        else if (id == "idate" ) {
            value <- "2018-09-01"
        }
        else {
            message("rciop.get_param:id not found...")
        }
        
        return (value)
    } # rciop.getparam

} # end stub

rciop.getparam() overridden by local stub function...


Add your code to test...
Remove current, or paste other code or add additional code blocks.

In [26]:
## Read the main input
## This is the reference link to the model configuration

# print
#rciop.log("INFO", paste("Processing input:", input1, sep=" "))

# Query the input reference
#message(paste("opensearch-client '",input1,"' enclosure", sep=""))
#opensearchCmd=paste("opensearch-client '",input1,"' enclosure")
#input_enclosure <- system(command = opensearchCmd,intern = T)
#rciop.log("INFO", input_enclosure)

# Download the file
#model_file <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

#if (model_file$exit.code==0) {
#    local.url <- model_file$output
#}

# Works for single file after unzip
#local.url # One file, complete path incl. filename
#my_data <- read.delim(local.url)
#print (my_data)

In [27]:
#par_model <- rciop.getparam("model")
#message(par_model)

In [39]:
## Read the main input
## This is the reference link to the model configuration

# print
rciop.log("INFO", paste("Processing input:", input2, sep=" "))

# Query the input reference
message(paste("opensearch-client '",input2,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",input2,"' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", input_enclosure)

# Download the dir(s)
model_config_dir <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

if (model_config_dir$exit.code==0) {
    local.model_config_dir <- model_config_dir$output
}
local.model_config_dir # File or directory, complete path incl.
# Content for now:
# info-forecast.txt - config file for Hype model

#if (local.model_config_dir == "" || local.model_config_dir == NULL) {
#    message("No model config path...")
#    q(99)
#}

model_config_file <- "dependencies.txt" # Shall be parsed to get different config items
# tmpdir/niger-hype-model-2.23/ # Currently for now:
# - dependencies.txt # Config for GFD-files etc.
# - info-forecast.txt # Hype config file


path_to_file <- paste(local.model_config_dir, model_config_file, sep="/")
path_to_file
# Read contents of file, handling separators ';'.
model_config_data <- read.csv2(path_to_file, header=FALSE, sep=";")
#class(model_config_data) # data.frame
#dim(model_config_data) # 6,3 (rows,cols)
# Add column names
names(model_config_data) <- c('subdir','url','querypattern','info')
model_config_data
# Select column by name
#model_config_data[,'query-pattern'] # all rows, col 3 ok
#model_config_data[,c('subdir','querypattern')] # all rows, multiple columns ok

#model_config_data$subdir
#model_config_data[3,'subdir']
# Iterate over each row to locate e.g. sub directory, od-daily, to use column data
for (r in 1:nrow(model_config_data)) { #  if statement below not works
    subdir <- model_config_data[r,'subdir']
    if (subdir == 'od-daily') {
        message(paste0("od-daily at row index:", r))
    }
    if (subdir == 'model-data') {
        message(paste0("model-data at row index:", r))
    }
}

opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' enclosure


[1] " -o /workspace/tmpdir https://store.terradue.com/hydro-smhi/models/niger-hype-model-2.23.zip"


[1] "/workspace/tmpdir/niger-hype-model-2.23"

[1] "/workspace/tmpdir/niger-hype-model-2.23/dependencies.txt"

subdir,url,querypattern,info
model-data,https://recast.terradue.com/t2api/search/hydro-smhi/models,?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2,# zip-file. niger-hype v2.23 and gfd obs.
model-data-old-url,https://store.terradue.com/hydro-smhi/fanfar/model/niger-hype,,# stop using. not zip-file. niger-hype v2.23 and gfd obs.
hydrogfdei,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdei,SMHI,operational]",# netcdf
hydrogfdod,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdod,SMHI,operational]",# netcdf
ei-monthly,https://catalog.terradue.com/hydro-smhi/description,"[ei,monthly,SMHI,operational]",# netcdf
od-daily,https://catalog.terradue.com/hydro-smhi/description,"[od,daily,SMHI,operational]",# netcdf
od-monthly,https://catalog.terradue.com/hydro-smhi/description,"[od,monthly,SMHI,operational]",# netcdf
ecoper,https://catalog.terradue.com/hydro-smhi/description,"[ecoper,SMHI,operational]",# netcdf


model-data at row index:1
od-daily at row index:6


In [40]:
## Read the Hype data dirs
## This code continues to retrieve the data dirs (rciop.copy)
## Other R code not yet updated to handle paths (file copy from tmp dir to run dir etc.) instead of urls.

# print
#rciop.log("INFO", paste("Processing config for model data:", input2, sep=" "))
rciop.log("INFO", "Processing config for model data")

rowIndex=0
for (r in 1:nrow(model_config_data)) {
    subdir <- model_config_data[r,'subdir']
    if (subdir == 'model-data') {
        rowIndex=r
        message(paste0("model-data at row index:", r))
    }
}
if (rowIndex == 0) {
    q(98)
}
#subdir <- model_config_data[rowIndex,'subdir']
url <- model_config_data[rowIndex,'url']
query <- model_config_data[rowIndex,'querypattern']
#comment <- model_config_data[rowIndex,'info']

# For now, if needed, pass url from model_config_data['model-data-old-url'] to let present R code continue rciop.copy locally.

# Query the input reference
message(paste("opensearch-client '",url,query,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",url,query,"' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", input_enclosure)

# Download the dir(s)
model_data_dirs <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

if (model_data_dirs$exit.code==0) {
    local.dir <- model_data_dirs$output
}
local.dir # File or directory, complete path incl.
# tmpdir/niger-hype-data/
# - data/gfd/archive/Pobs.txt, TMAXobs.txt, TMINobs.txt, Tobs.txt
# - v2.23/....... root dir for forcingarchive, shapefiles, statefiles etc.
# This/These path(s) shall be passed to getHypeAppSetup as dir, now urls that it currently downloads - now already downloaded...
# No file to parse etc. only pass path to the R-get functions


model-data at row index:1
opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2' enclosure


[1] " -o /workspace/tmpdir https://store.terradue.com/hydro-smhi/models/niger-hype-data.zip"


[1] "/workspace/tmpdir/niger-hype-data"

In [30]:
# opensearch for netcdf files:


In [38]:
# Query the input reference
#opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure,identifier")
# Gives a lot of urls to files, test with e.g. count=4 (for the 4 lastest?)

# When time to perform the download, only use enclosure skip identifier etc.
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=8' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure")
# Less? Yes, Correct? Count = 4 gives me urls/files for 2019-09-10 at this time, hmm...

# Search
input_enclosure <- system(command = opensearchCmd,intern = T) # Gives a lot of urls to files, test with e.g. count=4 (for the 4 lastest?)
# Print
message(input_enclosure)

if (length(input_enclosure >= 1)) {
    nc_tmpdir <- paste(TMPDIR,"nc-files",sep="/")
    dir.create(nc_tmpdir)
    for (url in 1:length(input_enclosure)) {
        rciop.copy(input_enclosure[url],nc_tmpdir) # Retreived 4 files, but incorrect date as seen before.
    }
}

https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmin_od-daily_20190910_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190910_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190910_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmax_od-daily_20190910_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmax_od-daily_20190909_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190909_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmin_od-daily_20190909_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190909_fanfar_SMHI.nc
Warning message in dir.create(nc_tmpdir):
“'/workspace/tmpdir/nc-files' already exists”

[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmin_od-daily_20190910_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190910_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190910_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmax_od-daily_20190910_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmax_od-daily_20190909_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190909_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/201